# MultiVelo Template

This is an example of basic workflow for 10X Cell Ranger ARC 2.0 output.
```
.
|-- MultiVelo_Template.ipynb
|-- outs
|   |-- analysis
|   |   `-- feature_linkage
|   |       `-- feature_linkage.bedpe
|   |-- filtered_feature_bc_matrix
|   |   |-- barcodes.tsv.gz
|   |   |-- features.tsv.gz
|   |   `-- matrix.mtx.gz
|   `-- atac_peak_annotation.tsv
|-- seurat_wnn
|   |-- nn_cells.txt
|   |-- nn_dist.txt
|   `-- nn_idx.txt
`-- velocyto
    `-- gex_possorted_bam_XXXXX.loom
```
Please replace ... with appropriate values.

In [ ]:
import os
import scipy
import numpy as np
import pandas as pd
import scanpy as sc
import scvelo as scv
import multivelo as mv
import matplotlib.pyplot as plt

In [ ]:
scv.settings.verbosity = 3
scv.settings.presenter_view = True
scv.set_figure_params('scvelo')
pd.set_option('display.max_columns', 100)
pd.set_option('display.max_rows', 200)
np.set_printoptions(suppress=True)

In [ ]:
# Read in RNA and filter
adata_rna = scv.read('velocyto/gex_possorted_bam_XXXXX.loom', cache=True)
adata_rna.obs_names = [x.split(':')[1][:-1] + '-1' for x in adata_rna.obs_names]
adata_rna.var_names_make_unique()
sc.pp.filter_cells(adata_rna, min_counts=...)
sc.pp.filter_cells(adata_rna, max_counts=...)

In [ ]:
# Read in ATAC, gene aggregate, and filter
adata_atac = sc.read_10x_mtx('outs/filtered_feature_bc_matrix/', var_names='gene_symbols', cache=True, gex_only=False)
adata_atac = adata_atac[:,adata_atac.var['feature_types'] == "Peaks"]

In [ ]:
# Aggregate peaks around each gene as well as those that have high correlations with promoter peak or gene expression
adata_atac = mv.aggregate_peaks_10x(adata_atac, 
                                    'outs/atac_peak_annotation.tsv', 
                                    'outs/analysis/feature_linkage/feature_linkage.bedpe', 
                                    verbose=True) 

In [ ]:
sc.pp.filter_cells(adata_atac, min_counts=...)
sc.pp.filter_cells(adata_atac, max_counts=...)

In [ ]:
# Find shared cells and genes between RNA and ATAC
shared_cells = pd.Index(np.intersect1d(adata_rna.obs_names, adata_atac.obs_names))
shared_genes = pd.Index(np.intersect1d(adata_rna.var_names, adata_atac.var_names))
len(shared_cells), len(shared_genes)

In [ ]:
adata_rna = adata_rna[shared_cells, shared_genes]
adata_atac = adata_atac[shared_cells, shared_genes]

In [ ]:
# Normalize RNA
scv.pp.filter_and_normalize(adata_rna, min_shared_counts=..., n_top_genes=...)

In [ ]:
# Optionally, regress out the effects of cell cycle and/or scale RNA matrix if it gives better clustering results
# scv.tl.score_genes_cell_cycle(adata_rna)
# sc.pp.regress_out(adata_rna, ['S_score', 'G2M_score’])
# sc.pp.scale(adata_rna)

In [ ]:
scv.pp.moments(adata_rna, n_pcs=..., n_neighbors=...)
scv.tl.umap(adata_rna) # compute UMAP embedding
sc.tl.leiden(adata_rna) # compute clusters

In [ ]:
# Identify cell types
new_cluster_names = [...]

In [ ]:
adata_rna.rename_categories('leiden', new_cluster_names) # annotate clusters

In [ ]:
scv.pl.umap(adata_rna, color='leiden')

In [ ]:
# Normalize ATAC and subset for the same set of cells and genes
mv.tfidf_norm(adata_atac)
adata_atac = adata_atac[adata_rna.obs_names, adata_rna.var_names]

In [ ]:
# Write out filtered cells and prepare to run Seurat WNN
adata_rna.obs_names.to_frame().to_csv('filtered_cells.txt', header=False, index=False) 

In [ ]:
# Run Seurat WNN (R script can be found on GitHub)

In [ ]:
# Back in python, load the neighbors
nn_idx = np.loadtxt("seurat_wnn/nn_idx.txt", delimiter=',')
nn_dist = np.loadtxt("seurat_wnn/nn_dist.txt", delimiter=',')
nn_cells = pd.Index(pd.read_csv("seurat_wnn/nn_cells.txt", header=None)[0])

In [ ]:
np.all(nn_cells == adata_atac.obs_names) # make sure cell names match

In [ ]:
# WNN smooth the gene aggregated ATAC matrix, resulting in a new Mc matrix in adata_atac.layers
mv.knn_smooth_chrom(adata_atac, nn_idx, nn_dist) 

In [ ]:
# Run MultiVelo main function
adata_result = mv.recover_dynamics_chrom(adata_rna,
                                         adata_atac,
                                         max_iter=5, # coordinate-descent like optimization
                                         init_mode="invert", # simple, invert, or grid
                                         verbose=False,
                                         parallel=True,
                                         save_plot=False,
                                         rna_only=False,
                                         fit=True,
                                         n_anchors=500,
                                         extra_color_key='leiden' # used if save_plot=True
                                        )
# Full argument list can be shown with help(mv.recover_dynamics_chrom)

In [ ]:
adata_result.write("multivelo_result.h5ad") # save the result

In [ ]:
# adata_result = sc.read_h5ad('multivelo_result.h5ad')

In [ ]:
mv.pie_summary(adata_result) # gene type chart

In [ ]:
mv.switch_time_summary(adata_result) # switch time statistics

In [ ]:
mv.likelihood_plot(adata_result) # likelihood and model parameter statistics

In [ ]:
mv.velocity_graph(adata_result)
mv.latent_time(adata_result)

In [ ]:
mv.velocity_embedding_stream(adata_result, basis='umap', color='leiden') # velocity streams

In [ ]:
scv.pl.scatter(adata_result, color='latent_time', color_map='gnuplot', size=80) # latent time prediction

In [ ]:
# Some genes of interest
gene_list = [...]

In [ ]:
# Plot accessbility and expression against gene time or global latent time
mv.dynamic_plot(adata_result, gene_list, color_by='state', gene_time=True, axis_on=False, frame_on=False)

In [ ]:
# Phase portraits on the u-s, c-u, or 3-dimensional planes can be plotted
mv.scatter_plot(adata_result, gene_list, color_by='leiden', by='us', axis_on=False, frame_on=False) 